### Init (You'll need a .env file)

In [81]:
inputFileRoot = "PoC-v2-Data"
actuallySaveData = True

import os
import sys
import re
from typing import List, Dict, Any
from langchain_core.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from openai import OpenAIError
from dotenv import load_dotenv
from tabulate import tabulate
from IPython.display import HTML, display
from openpyxl import Workbook, reader, load_workbook
import pandas as pd

# The API key belongs in the .env file
loaded = load_dotenv()

from langchain.globals import set_verbose, set_debug
set_debug(True)
set_verbose(True)

### Find the most recent data file

In [82]:
# Example usage:
#filename = "base.123.xlsx"
#new_filename = increment_version(filename)
#print(new_filename)  # Output: base.124.xlsx
def increment_version(filename):
    try:
        root, version, extension = filename.split('.')
        version = int(version) + 1
        return f"{root}.{version}.{extension}"
    except ValueError:
        raise ValueError("Filename format should be 'root.version.extension' and version should be an integer")

# Example usage:
#directory = "."
#root_segment = "PoC-v2-Data"
#highest_version_file = get_highest_version_file(directory, root_segment)
#print(highest_version_file)
def get_highest_version_file(directory, root_segment):
    highest_version = -1
    highest_version_file = None
    
    pattern = re.compile(rf'^{root_segment}\.(\d+)\.xlsx$')
    
    for filename in os.listdir(directory):
        match = pattern.match(filename)
        if match:
            version = int(match.group(1))
            if version > highest_version:
                highest_version = version
                highest_version_file = filename
    
    return highest_version_file

latestDataFileName = get_highest_version_file(".", inputFileRoot)

### Load Business Requirements from the spreadsheet

In [83]:
def get_BReqs_from_ws() -> pd.DataFrame:
    wb = reader.excel.load_workbook(filename=latestDataFileName, read_only=True, keep_vba=False, data_only=False, keep_links=True, rich_text=False)
    ws_br = wb["BReqs"]
    df = pd.DataFrame(ws_br.values)
    wb.close()
    df = df.rename(columns={0: "Name", 1: "Description"})
    #print(df.head())
    return df

def get_BReq_context():
    df = get_BReqs_from_ws()
    result = ""
    for index, row in df.iterrows():
        result += "Requirement-" + str(row["Name"]) + " is that " + str(row["Description"]) + ". "
    return result

### Generate some stories

In [84]:
class Story(BaseModel):
    title: str = Field(description="The title of the story")
    story_text: str = Field(description="The user story in the 'As a... I want to... So That...' format.")
    relevance_score: float = Field(description="The relevance of the story to the scenario. 0 is the worst, 10 is the best.")

class Stories(BaseModel):
    contents: List[Story] = Field(description="A list of Stories")

def create_chat_model(temperature: float = 0.0) -> ChatOpenAI:
    try:
        return ChatOpenAI(temperature=temperature)
    except OpenAIError as e:
        print(f"Error creating ChatOpenAI model: {e}")
        sys.exit(1)

def create_parser() -> PydanticOutputParser:
    return PydanticOutputParser(pydantic_object=Stories)

def create_chat_prompt(template: str) -> ChatPromptTemplate:
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    return ChatPromptTemplate.from_messages([system_message_prompt])

def generate_stories(model: ChatOpenAI, chat_prompt: ChatPromptTemplate, parser: PydanticOutputParser,
                   input_data: Dict[str, Any]) -> Stories:
    try:
        print("***Chat Prompt***")
        print(chat_prompt)
        print("***Input Data***")
        print(input_data)
        prompt_and_model = chat_prompt | model
        result = prompt_and_model.invoke(input_data)
        return parser.parse(result.content)
    except OpenAIError as e:
        print(f"Error generating names: {e}")
        return Stories(contents=[])
    except ValueError as e:
        print(f"Error parsing output: {e}")
        print(f"The actual response was {result}")
        return Stories(contents=[])

def get_stories(feature: str):
    if 'OPENAI_API_KEY' not in os.environ:
        print("Error: OPENAI_API_KEY environment variable is not set.")
        sys.exit(1)

    breqs = get_BReq_context()
    print(breqs)
    
    principles = """
    - The story should evoke the core goals of users of the {feature} feature.
    - The title should be between 3 and 8 words long.
    - The user story must be in the 'As a… I want to… So That…' format. 
    - Only stories that do not conflict with the following hard requirements may be generated:
    {breqs}
    
    """

    template = """Generate the 10 most relevent user stories for a new custom software capability that offers a {feature} feature.
    You must follow the following principles: {principles}
    {format_instructions}
    """

    result = Stories(contents=[])
    try:
        model = create_chat_model()
        parser = create_parser()
        chat_prompt = create_chat_prompt(template)

        input_data = {
            "breqs": breqs,
            "principles": principles,
            "feature": feature,
            "format_instructions": parser.get_format_instructions(),
        }
        #print(input_data)

        result = generate_stories(model, chat_prompt, parser, input_data)
        return result.contents
        
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        sys.exit(1)

### Run the procs so far

In [85]:
stories = get_stories("An online shopping cart")
#print(stories)

Requirement-Cart-Security is that Users must not be permitted to save their cart for later.. 
***Chat Prompt***
input_variables=['feature', 'format_instructions', 'principles'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['feature', 'format_instructions', 'principles'], template='Generate the 10 most relevent user stories for a new custom software capability that offers a {feature} feature.\n    You must follow the following principles: {principles}\n    {format_instructions}\n    '))]
***Input Data***
{'breqs': 'Requirement-Cart-Security is that Users must not be permitted to save their cart for later.. ', 'principles': "\n    - The story should evoke the core goals of users of the {feature} feature.\n    - The title should be between 3 and 8 words long.\n    - The user story must be in the 'As a… I want to… So That…' format. \n    - Only stories that do not conflict with the following hard requirements may be generated:\n    {breqs}\n    \n    ", 'feat

In [86]:
stories_dicts = [story.__dict__ for story in stories]
table = tabulate(stories_dicts, tablefmt='html', headers="keys")
display(HTML(table))

### Save the new stories to the 'Stories' worksheet

In [87]:
def save_stories_to_excel(src_file_path: str, dst_file_path: str, sheet_name: str, stories: Stories):
    # Open the workbook and select the worksheet
    workbook = load_workbook(src_file_path)
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.create_sheet(sheet_name)

    # Delete all rows below the header
    sheet.delete_rows(2, sheet.max_row)

    # Define the headers
    #headers = ["Title", "Story Text", "Relevance Score"]
    #sheet.append(headers)
    
    # Write the data to the worksheet
    for story in stories:
        sheet.append([story.title, story.story_text, story.relevance_score])
    
    # Save the workbook
    workbook.save(dst_file_path)
    workbook.close()

# Save these stories to an Excel file
nextFileName = increment_version(latestDataFileName)
if actuallySaveData:
    save_stories_to_excel(latestDataFileName, nextFileName, "Stories", stories)

In [88]:
xls = pd.ExcelFile(nextFileName)
for sheet_name in xls.sheet_names:
    df = pd.read_excel(nextFileName, sheet_name=sheet_name)
    print(f"Sheet name: {sheet_name}")
    display(df)
xls.close()

Sheet name: BReqs


,Cart-Security,Users must not be permitted to save their cart for later.


Sheet name: Features


""


Sheet name: Stories


,Title,Story Text,Relevance Score
